In [94]:
#Pacotes
import pandas as pd
import numpy
from sklearn.ensemble import RandomForestClassifier



In [95]:
#Importando base de dados: German Credit Data
credit_data=pd.read_csv("credit_data.csv",skiprows=0)
credit_data;
credit_data["class"]=credit_data["class"]-1

In [96]:
#The creation of dummy variables step is similar to the logistic regression preprocessing step:
dummy_stseca = pd.get_dummies(credit_data['Status_of_existing_checking_account'],prefix='status_exs_accnt')
dummy_ch = pd.get_dummies(credit_data['Credit_history'],
prefix='cred_hist')
dummy_purpose = pd.get_dummies(credit_data['Purpose'],prefix='purpose')
dummy_savacc = pd.get_dummies(credit_data["Savings_account/bonds"],prefix='sav_acc')
dummy_presc = pd.get_dummies(credit_data["Present_employment_since"],
prefix='pre_emp_snc')
dummy_perssx = pd.get_dummies(credit_data["Personal_status_and_sex"],
prefix='per_stat_sx')
dummy_othdts = pd.get_dummies(credit_data["Other_debtors_/_guarantors"],
prefix='oth_debtors')
dummy_property = pd.get_dummies(credit_data['Property'],
prefix='property')
dummy_othinstpln =pd.get_dummies(credit_data['Other_installment_plans'],prefix='oth_inst_pln')
dummy_housing = pd.get_dummies(credit_data["Housing"],prefix='housing')
dummy_job = pd.get_dummies(credit_data["Job"], prefix='job')
dummy_telephn = pd.get_dummies(credit_data["Telephone"],prefix='telephn')
dummy_forgnwrkr = pd.get_dummies(credit_data["foreign_worker"],prefix='forgn_wrkr')
 

In [97]:
continuous_columns = ['Duration_in_month', 'Credit_amount','Installment_rate_in_percentage_of_disposable_income',
'Present_residence_since','Age_in_years','Number_of_existing_credits_at_this_bank','Number_of_people_being_liable_to_provide_maintenance_for']


In [98]:
credit_continuous= credit_data[continuous_columns]


In [100]:
credit_data_new = pd.concat([dummy_stseca, dummy_ch,dummy_purpose,dummy_savacc,dummy_presc,dummy_perssx,dummy_othdts, dummy_property,
dummy_othinstpln,dummy_housing,dummy_job, dummy_telephn, dummy_forgnwrkr,credit_continuous,credit_data["class"]],axis=1)

In [104]:
rf_fit = RandomForestClassifier( n_estimators=1000, criterion="gini", max_depth=100, min_samples_split=3,min_samples_leaf=2)




In [107]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split( credit_data_new.drop(["class"],axis=1),credit_data_new["class"],train_size =
0.7,random_state=42)

rf_fit.fit(x_train,y_train)

RandomForestClassifier(max_depth=100, min_samples_leaf=2, min_samples_split=3,
                       n_estimators=1000)

In [111]:
print ("\nRandom Forest -Train Confusion Matrix\n\n",pd.crosstab(y_train, rf_fit.predict( x_train),rownames =
["Actuall"],colnames = ["Predicted"]))

from sklearn.metrics import accuracy_score,classification_report

print ("\n Random Forest - Train accuracy",round(accuracy_score(y_train, rf_fit.predict(x_train)),3))



Random Forest -Train Confusion Matrix

 Predicted    0    1
Actuall            
0          491    0
1           32  177

 Random Forest - Train accuracy 0.954


In [112]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV

In [115]:
pipeline = Pipeline([("clf",RandomForestClassifier(criterion="gini"))])

parameters = {"clf__n_estimators":(1000,2000,3000),
  "clf__max_depth":(100,200,300),
  "clf__min_samples_split":(2,3),
  "clf__min_samples_leaf":(1,2) }

In [125]:
grid_search = GridSearchCV(pipeline,parameters,n_jobs=-1, cv=5,verbose=1,scoring="accuracy")

grid_search.fit(x_train,y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('clf', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'clf__max_depth': (100, 200, 300),
                         'clf__min_samples_leaf': (1, 2),
                         'clf__min_samples_split': (2, 3),
                         'clf__n_estimators': (1000, 2000, 3000)},
             scoring='accuracy', verbose=1)

In [130]:
print ('Best Training score: %0.3f' % grid_search.best_score_)
print ('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print ('\t%s: %r' % (param_name, best_parameters[param_name]))




Best Training score: 0.773
Best parameters set:
	clf__max_depth: 300
	clf__min_samples_leaf: 2
	clf__min_samples_split: 3
	clf__n_estimators: 1000


In [135]:
predictions = grid_search.predict(x_test)
print ("Testing accuracy:",round(accuracy_score(y_test,
predictions),4))

Testing accuracy: 0.7567


In [140]:
print ("\nComplete report of Testing data\n",classification_report(y_test,predictions))



Complete report of Testing data
               precision    recall  f1-score   support

           0       0.76      0.95      0.84       209
           1       0.72      0.32      0.44        91

    accuracy                           0.76       300
   macro avg       0.74      0.63      0.64       300
weighted avg       0.75      0.76      0.72       300



In [141]:
 print ("\n\nRandom Forest Grid Search- Test Confusion Matrix\n\n",pd.crosstab(y_test, predictions,rownames = ["Actuall"],colnames =
["Predicted"]))



Random Forest Grid Search- Test Confusion Matrix

 Predicted    0   1
Actuall           
0          198  11
1           62  29
